<a href="https://colab.research.google.com/github/tmandingwa/tmandingwa/blob/main/Prac_Transportation_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Network Models
We have a two key learning points in this session.

a. Figuring out how to set up a network model

b. Network models usually entail many parameters and constraints, which are impossible to enter manually. So usually we have those parameters in a table. We will learn how to

  i. Input / output files into and out of a colab environment (skip a step if you are using local python)

  ii. Create a dataframe for the parameters.

  iii. Input the table of parameters into the model.

In [1]:
# The usual installation of packages.
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
solver = 'glpk'

# Import the environment
import pandas as pd
import pyomo.environ as pyo

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 126455 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

# 2 Expected solution of the transportation model

## 2.1 Import parameters from file

In [2]:
# Upload a file in your local machine to Colab
# Click Browse. Then select the file transport_data2025.xlsx
# Inspect if the file exists in the "content" folder.

from google.colab import files
uploaded = files.upload()

# ALTERNATIVE: 1. Click on folder icon on the left; 2. Click upload

Saving transport_data2025.xlsx to transport_data2025.xlsx


In [3]:
# Read the file
file = 'transport_data2025.xlsx'

# Index_col defines the searchable "index"
Costs = pd.read_excel(file, sheet_name='TransportCosts', index_col=0, usecols="A:E", nrows=4)
StoreReqs = pd.read_excel(file, sheet_name='Requirements', index_col=0, usecols="A:B", nrows=5)
FactoryCaps = pd.read_excel(file, sheet_name='Capacity', index_col=0, usecols="A:B", nrows=4)

In [4]:
# Retrieving specific values
print(StoreReqs.index)
print(StoreReqs.values)
print(FactoryCaps.index)
print(FactoryCaps.values)
print(Costs.at['Minneapolis', 'Atlanta'])


Index(['Atlanta', 'Boston', 'Chicago', 'Denver'], dtype='object', name='StoreLocation')
[[7500]
 [8500]
 [9500]
 [8000]]
Index(['Minneapolis', 'Pittsburgh', 'Tucson'], dtype='object', name='FactoryLocation')
[[ 9000]
 [12000]
 [13000]]
0.6


In [7]:
# Declare model
model = pyo.ConcreteModel("Transport Model")

## 2.2 Set of decision variables

In [8]:
model.factories = pyo.Set(initialize=FactoryCaps.index)
# [TO ADD: model.stores = ...]


# Generate a set of variables of size model.factories x model.stores
model.x = pyo.Var(model.factories, model.stores, domain=pyo.NonNegativeReals)

print(len(model.factories))
print(len(model.stores))
print(len(model.x))

AttributeError: 'ConcreteModel' object has no attribute 'stores'

## 2.3 Objective function

In [9]:
model.cost = pyo.Objective(
    expr=sum(Costs.at[i, j] * model.x[i, j] for i in model.factories for j in model.stores),
    sense=pyo.minimize
)

AttributeError: 'ConcreteModel' object has no attribute 'stores'

## 2.4 Constraints

In [ ]:
# Factory capacity constraints
model.factory_constraints = pyo.ConstraintList()
for i in model.factories:
    model.factory_constraints.add(sum(model.x[i, j] for j in model.stores) <= FactoryCaps.at[i, 'Capacity'])

# [TO ADD:] Requirement constraints

## 2.5 Solve

In [ ]:
# Solve the model (assuming you've added constraints)
pyo.SolverFactory(solver).solve(model)
print(f"Objective value: {pyo.value(model.cost)}")

# Output results
df_assign = pd.DataFrame(columns=model.stores, index=model.factories)

for i in model.factories:
    for j in model.stores:
        print(f"x[{i}, {j}] = {model.x[i, j].value}")
        df_assign.at[i,j] = model.x[i, j]()

df_assign.to_excel("transport_output.xlsx", index=True)

df_assign

Objective value: 12025.0
x[Minneapolis, Atlanta] = 0.0
x[Minneapolis, Boston] = 0.0
x[Minneapolis, Chicago] = 9000.0
x[Minneapolis, Denver] = 0.0
x[Pittsburgh, Atlanta] = 3500.0
x[Pittsburgh, Boston] = 8500.0
x[Pittsburgh, Chicago] = 0.0
x[Pittsburgh, Denver] = 0.0
x[Tucson, Atlanta] = 4000.0
x[Tucson, Boston] = 0.0
x[Tucson, Chicago] = 500.0
x[Tucson, Denver] = 8000.0


,Atlanta,Boston,Chicago,Denver
Minneapolis,0.0,0.0,9000.0,0.0
Pittsburgh,3500.0,8500.0,0.0,0.0
Tucson,4000.0,0.0,500.0,8000.0


In [ ]:
# Output file stored in colab to your local machine

from google.colab import files

files.download('transport_output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>